### Importing Packages

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import geopandas as pdg
from zipfile import ZipFile
from shapely.geometry import box
from matplotlib.colors import ListedColormap

In [4]:
def calc_length(roads):

    """"
    calculates the lengths of the road segments in roads

    paramter roads: A geopandas.geodataframe.GeoDataFrame
    """
    
    lengths = roads.length
    lengths = lengths/1000
    return lengths

In [ ]:
# NewPHH Development Algorithm 1.3V -- section 2 -- 3c
# - PRUID corresponds to specific provinces 
# - use a subsection of a provice (PRUID)
# - use PEI with 11 or first 20 
# - see if I can do overlay with the CCB_Shape 
# - pick new brunswick nexy sfter PEI for extra validfation of the dataset

In [5]:
# Importing CCB_Shape file
# - CCB_Shape = contains a polygon showing the areas where we have Ecopia Data
 
with ZipFile("C:/Users/daniel.kim/DataSet/CCB_Shape.zip", "r") as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.startswith("CCB"):
           zipObj.extract(fileName, "CCB_Shape_unzipped") 

CCB_Shape_File_data_set = pdg.read_file("C:/Users/daniel.kim/Pytest/CCB_Shape_unzipped/CCB_Shape/shape3.shp")

In [6]:
# Importing Stats Canada 2021 Dissemination Block Boundary file
# - ldb_000a21a_e = block boundary file
# - better to use functions 
# - road segment functions is a god example 

with ZipFile("C:/Users/daniel.kim/DataSet/ldb_000a21a_e.zip", "r") as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName.startswith("ldb_000a21a_e"):
           zipObj.extract(fileName, "ldb_000a21a_e_unzipped")

ldb_000a21a_e_data_set = pdg.read_file("C:/Users/daniel.kim/Pytest/ldb_000a21a_e_unzipped/ldb_000a21a_e.shp")

# Working with a smaller subset of the boundary file: PEI (PRUID == 11)
subSet_PEI_11 = ldb_000a21a_e_data_set[ldb_000a21a_e_data_set["PRUID"] == "11"].copy()
subSet_PEI_11


,DBUID,DGUID,DBRPLAMX,DBRPLAMY,LANDAREA,PRUID,geometry
0,11030266006,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((8318584.069 1624827.580, 8318572.731..."
1,11030175002,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((8324381.409 1624971.783, 8324287.837..."
2,11020453003,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((8329318.634 1623843.626, 8329316.971..."
3,11030266007,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((8316701.251 1623784.680, 8316732.303..."
4,11030267001,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((8321363.814 1624751.220, 8321363.109..."
...,...,...,...,...,...,...,...
492267,11030122012,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((8312743.754 1612185.426, 8312646.249..."
495177,11030289033,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((8322957.594 1606642.777, 8322964.940..."
495178,11030291011,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((8322386.503 1610761.511, 8322526.203..."
495179,11030291012,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((8324756.497 1612164.654, 8324655.857..."


In [7]:
# Using Spatial Joins for the CCB file (CCB_Shape_File_data_set) and Block Boundary file (ldb_000a21a_e_data_set)
# - join the geometry of the two files
# - store the geospatial 
# - message is saying that the tyeo files are using systems
# - 39, 79 make sure all geo spatial file is in the same coordinate system 
# - sjoin might not be the best way to do this, sjoin will not give the overlay
# - area might be in square meters, can compre in square kilos
# - care about what is overlapping and not overlapping 
# - use overlay and calculate the area
# - start off with a smaller subset 

# Using the subset of PEI with spatial join with the CC_B_Shape file
subSet_PEI_11_GeoUpdate = subSet_PEI_11.to_crs(3979)
CCB_Shape_GeoUpdate = CCB_Shape_File_data_set.to_crs(3979)

# joining the updated geometry of the subset boundary file with the updated geometry of CCB_Shape file
overlay_CCB_PEI_Boundary = pdg.overlay(CCB_Shape_GeoUpdate, subSet_PEI_11_GeoUpdate, how="intersection")

In [47]:
# calculating the intersected areas from overlaying_CCB_PEIBoundary and creating a new column called "UPDATED_AREA"

# overlaying_CCB_PEIBoundary["AREA"] = overlaying_CCB_PEIBoundary.geometry.area
overlay_CCB_PEI_Boundary["OVERLAY_AREA"] = overlay_CCB_PEI_Boundary.apply(lambda row: row.geometry.area, axis=1)

# use a for loop to update the values in each row of the AREA column in both the overlaying_CCB_PEIBoundary df 
for i in range(len(overlay_CCB_PEI_Boundary)):
    overlay_CCB_PEI_Boundary.loc[i, "OVERLAY_AREA"] * 1000000
    
# Calculating the area of the original Stats Canada Dissemination Block (PEI) 
subSet_PEI_11_GeoUpdate["PEI_TOTAL_AREA"] = subSet_PEI_11_GeoUpdate.apply(lambda row: row.geometry.area, axis=1)

# use a for loop to update the values in each row of the AREA column in both the subSet_PEI_11_GeoUpdate df 
# for i in range(len(subSet_PEI_11_GeoUpdate)):
#    subSet_PEI_11_GeoUpdate.loc[i, "PEI_TOTAL_AREA"] * 1000000
# for i in range(len(subSet_PEI_11_GeoUpdate)):
#    subSet_PEI_11_GeoUpdate.loc[i, "PEI_TOTAL_AREA"] * 1000000

In [26]:
subSet_PEI_11_GeoUpdate

,DBUID,DGUID,DBRPLAMX,DBRPLAMY,LANDAREA,PRUID,geometry,PEI_TOTAL_AREA
0,11030266006,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335549.465 305570.376, 2335536.420 ...",6.438442e+05
1,11030175002,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341332.671 305999.874, 2341238.622 ...",1.204210e+05
2,11020453003,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346319.460 305116.202, 2346317.175 ...",9.319654e+05
3,11030266007,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((2333720.286 304436.028, 2333718.190 ...",1.004782e+06
4,11030267001,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((2338329.598 305630.988, 2338328.101 ...",2.065565e+05
...,...,...,...,...,...,...,...,...
492267,11030122012,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((2330338.755 292655.972, 2330242.422 ...",8.515808e+04
495177,11030289033,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((2340813.133 287622.993, 2340820.509 ...",4.689646e+05
495178,11030291011,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((2340039.921 291708.609, 2340181.797 ...",2.781303e+06
495179,11030291012,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((2342337.948 293226.752, 2342241.610 ...",3.267876e+06


In [43]:
overlay_CCB_PEI_Boundary  

,id,DBUID,DGUID,DBRPLAMX,DBRPLAMY,LANDAREA,PRUID,geometry,OVERLAY_AREA
0,1.0,11030266006,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335536.420 305604.721, 2335509.892 ...",6.438442e+05
1,1.0,11030175002,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341238.622 306007.263, 2341189.666 ...",1.204210e+05
2,1.0,11020453003,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346317.175 305128.782, 2346293.754 ...",9.319654e+05
3,1.0,11030266007,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((2333718.190 305109.136, 2333707.787 ...",1.004782e+06
4,1.0,11030267001,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((2338328.101 305647.033, 2338321.230 ...",2.065565e+05
...,...,...,...,...,...,...,...,...,...
2737,1.0,11030122012,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((2330242.422 292629.768, 2330227.732 ...",8.515808e+04
2738,1.0,11030289033,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((2340820.509 287622.558, 2340840.959 ...",4.689646e+05
2739,1.0,11030291011,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((2340181.797 291667.920, 2340277.515 ...",2.781303e+06
2740,1.0,11030291012,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((2342241.610 293137.011, 2341862.109 ...",3.267876e+06


In [16]:
# Merging PEI and CCB together through DBUID
MERGED_DF_CCB_PEI = pd.merge(overlay_CCB_PEI_Boundary, subSet_PEI_11_GeoUpdate, on = ['DBUID'], how = 'left')
MERGED_DF_CCB_PEI


,id,DBUID,DGUID_x,DBRPLAMX_x,DBRPLAMY_x,LANDAREA_x,PRUID_x,geometry_x,OVERLAY_AREA,DGUID_y,DBRPLAMX_y,DBRPLAMY_y,LANDAREA_y,PRUID_y,geometry_y,PEI_TOTAL_AREA
0,1.0,11030266006,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335536.420 305604.721, 2335509.892 ...",6.438442e+05,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335549.465 305570.376, 2335536.420 ...",6.438442e+05
1,1.0,11030175002,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341238.622 306007.263, 2341189.666 ...",1.204210e+05,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341332.671 305999.874, 2341238.622 ...",1.204210e+05
2,1.0,11020453003,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346317.175 305128.782, 2346293.754 ...",9.319654e+05,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346319.460 305116.202, 2346317.175 ...",9.319654e+05
3,1.0,11030266007,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((2333718.190 305109.136, 2333707.787 ...",1.004782e+06,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((2333720.286 304436.028, 2333718.190 ...",1.004782e+06
4,1.0,11030267001,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((2338328.101 305647.033, 2338321.230 ...",2.065565e+05,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((2338329.598 305630.988, 2338328.101 ...",2.065565e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2737,1.0,11030122012,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((2330242.422 292629.768, 2330227.732 ...",8.515808e+04,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((2330338.755 292655.972, 2330242.422 ...",8.515808e+04
2738,1.0,11030289033,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((2340820.509 287622.558, 2340840.959 ...",4.689646e+05,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((2340813.133 287622.993, 2340820.509 ...",4.689646e+05
2739,1.0,11030291011,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((2340181.797 291667.920, 2340277.515 ...",2.781303e+06,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((2340039.921 291708.609, 2340181.797 ...",2.781303e+06
2740,1.0,11030291012,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((2342241.610 293137.011, 2341862.109 ...",3.267876e+06,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((2342337.948 293226.752, 2342241.610 ...",3.267876e+06


In [17]:
# Calculating the non-ecopia area
MERGED_DF_CCB_PEI['NON_ECOPIA_AREA'] = MERGED_DF_CCB_PEI['PEI_TOTAL_AREA'] - MERGED_DF_CCB_PEI['OVERLAY_AREA']
MERGED_DF_CCB_PEI

,id,DBUID,DGUID_x,DBRPLAMX_x,DBRPLAMY_x,LANDAREA_x,PRUID_x,geometry_x,OVERLAY_AREA,DGUID_y,DBRPLAMX_y,DBRPLAMY_y,LANDAREA_y,PRUID_y,geometry_y,PEI_TOTAL_AREA,NON_ECOPIA_AREA
0,1.0,11030266006,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335536.420 305604.721, 2335509.892 ...",6.438442e+05,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335549.465 305570.376, 2335536.420 ...",6.438442e+05,2.328306e-10
1,1.0,11030175002,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341238.622 306007.263, 2341189.666 ...",1.204210e+05,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341332.671 305999.874, 2341238.622 ...",1.204210e+05,5.820766e-11
2,1.0,11020453003,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346317.175 305128.782, 2346293.754 ...",9.319654e+05,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346319.460 305116.202, 2346317.175 ...",9.319654e+05,0.000000e+00
3,1.0,11030266007,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((2333718.190 305109.136, 2333707.787 ...",1.004782e+06,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((2333720.286 304436.028, 2333718.190 ...",1.004782e+06,2.328306e-10
4,1.0,11030267001,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((2338328.101 305647.033, 2338321.230 ...",2.065565e+05,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((2338329.598 305630.988, 2338328.101 ...",2.065565e+05,2.910383e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2737,1.0,11030122012,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((2330242.422 292629.768, 2330227.732 ...",8.515808e+04,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((2330338.755 292655.972, 2330242.422 ...",8.515808e+04,-1.455192e-11
2738,1.0,11030289033,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((2340820.509 287622.558, 2340840.959 ...",4.689646e+05,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((2340813.133 287622.993, 2340820.509 ...",4.689646e+05,-1.746230e-10
2739,1.0,11030291011,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((2340181.797 291667.920, 2340277.515 ...",2.781303e+06,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((2340039.921 291708.609, 2340181.797 ...",2.781303e+06,0.000000e+00
2740,1.0,11030291012,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((2342241.610 293137.011, 2341862.109 ...",3.267876e+06,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((2342337.948 293226.752, 2342241.610 ...",3.267876e+06,-4.656613e-10


In [50]:
# Caculating the percentage of area overlap for both the Ecopia and non-Ecopia

# Percentage of non-overlapping Ecopia Data
MERGED_DF_CCB_PEI['PERCENTAGE_NOT_CONTAINING_ECOPIA'] = MERGED_DF_CCB_PEI['NON_ECOPIA_AREA'] / MERGED_DF_CCB_PEI['PEI_TOTAL_AREA']

# Percentage of overlapping Ecopia Data
MERGED_DF_CCB_PEI['PERCENTAGE_CONTAINING_ECOPIA'] = MERGED_DF_CCB_PEI['OVERLAY_AREA'] / MERGED_DF_CCB_PEI['PEI_TOTAL_AREA']

# Calculating the difference from the two polygons
MERGED_DF_CCB_PEI['DIff_FROM_OVERLAY_TO_PEI'] = MERGED_DF_CCB_PEI['geometry_x'].difference(MERGED_DF_CCB_PEI['geometry_y'], align = False)

# Checking two columns created
MERGED_DF_CCB_PEI

,id,DBUID,DGUID_x,DBRPLAMX_x,DBRPLAMY_x,LANDAREA_x,PRUID_x,geometry_x,OVERLAY_AREA,DGUID_y,DBRPLAMX_y,DBRPLAMY_y,LANDAREA_y,PRUID_y,geometry_y,PEI_TOTAL_AREA,NON_ECOPIA_AREA,PERCENTAGE_NOT_CONTAINING_ECOPIA,PERCENTAGE_CONTAINING_ECOPIA,DIff_FROM_OVERLAY_TO_PEI
0,1.0,11030266006,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335536.420 305604.721, 2335509.892 ...",6.438442e+05,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335549.465 305570.376, 2335536.420 ...",6.438442e+05,2.328306e-10,3.616258e-16,1.0,POLYGON EMPTY
1,1.0,11030175002,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341238.622 306007.263, 2341189.666 ...",1.204210e+05,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341332.671 305999.874, 2341238.622 ...",1.204210e+05,5.820766e-11,4.833681e-16,1.0,POLYGON EMPTY
2,1.0,11020453003,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346317.175 305128.782, 2346293.754 ...",9.319654e+05,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346319.460 305116.202, 2346317.175 ...",9.319654e+05,0.000000e+00,0.000000e+00,1.0,POLYGON EMPTY
3,1.0,11030266007,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((2333718.190 305109.136, 2333707.787 ...",1.004782e+06,2021S051311030266007,8.317127e+06,1.624551e+06,0.9840,11,"POLYGON ((2333720.286 304436.028, 2333718.190 ...",1.004782e+06,2.328306e-10,2.317225e-16,1.0,POLYGON EMPTY
4,1.0,11030267001,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((2338328.101 305647.033, 2338321.230 ...",2.065565e+05,2021S051311030267001,8.321554e+06,1.625145e+06,0.2023,11,"POLYGON ((2338329.598 305630.988, 2338328.101 ...",2.065565e+05,2.910383e-11,1.409001e-16,1.0,POLYGON EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2737,1.0,11030122012,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((2330242.422 292629.768, 2330227.732 ...",8.515808e+04,2021S051311030122012,8.312720e+06,1.612323e+06,0.0833,11,"POLYGON ((2330338.755 292655.972, 2330242.422 ...",8.515808e+04,-1.455192e-11,-1.708812e-16,1.0,POLYGON EMPTY
2738,1.0,11030289033,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((2340820.509 287622.558, 2340840.959 ...",4.689646e+05,2021S051311030289033,8.321665e+06,1.605450e+06,0.4585,11,"POLYGON ((2340813.133 287622.993, 2340820.509 ...",4.689646e+05,-1.746230e-10,-3.723586e-16,1.0,POLYGON EMPTY
2739,1.0,11030291011,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((2340181.797 291667.920, 2340277.515 ...",2.781303e+06,2021S051311030291011,8.321991e+06,1.610329e+06,2.5950,11,"POLYGON ((2340039.921 291708.609, 2340181.797 ...",2.781303e+06,0.000000e+00,0.000000e+00,1.0,POLYGON EMPTY
2740,1.0,11030291012,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((2342241.610 293137.011, 2341862.109 ...",3.267876e+06,2021S051311030291012,8.325367e+06,1.613422e+06,3.1967,11,"POLYGON ((2342337.948 293226.752, 2342241.610 ...",3.267876e+06,-4.656613e-10,-1.424966e-16,1.0,POLYGON EMPTY


In [51]:
MERGED_DF_CCB_PEI['DIff_FROM_OVERLAY_TO_PEI'].unique()

<GeometryArray>
[          <shapely.geometry.polygon.Polygon object at 0x000001F4742C7040>,
 ...
Length: 213, dtype: geometry

In [ ]:
# Mkaing sure all the values are unique
MERGED_DF_CCB_PEI["PERCENTAGE_CONTAINING_ECOPIA"].unique()

In [20]:
# Importing DB_2021 csv file 
DB_2021_CSV_File = pd.read_csv(r'C:\Users\daniel.kim\DataSet\DB_2021.csv')
DB_2021_CSV_File

,DBuid,DBdguid,DBpop_2021,DBtdwell_2021,DBurdwell_2021,DBarea,DBir_2021,PRuid,DBcode,CDcode,...,DAcode,FEDcode,ERcode,CCScode,CSDcode,CMAuid,CTname,CTcode,POPCTRRAuid,DPLcode
0,62080028013,2021S051362080028013,89.0,18.0,16.0,0.0226,0,62,13,8,...,28,1,10,98,81,0,9962.00,9962,1573,9962
1,62080028012,2021S051362080028012,103.0,16.0,16.0,0.0461,0,62,12,8,...,28,1,10,98,81,0,9962.00,9962,1573,9962
2,62080028011,2021S051362080028011,55.0,9.0,9.0,0.0731,0,62,11,8,...,28,1,10,98,81,0,9962.00,9962,1573,9962
3,62080028010,2021S051362080028010,42.0,8.0,7.0,1.2254,0,62,10,8,...,28,1,10,98,81,0,9962.00,9962,9962,9962
4,62080028009,2021S051362080028009,75.0,26.0,19.0,2.4607,0,62,9,8,...,28,1,10,98,81,0,9962.00,9962,9962,9962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498781,10010166001,2021S051310010166001,80.0,41.0,35.0,0.0207,0,10,1,1,...,166,7,10,519,519,1,170.02,4738,792,9910
498782,10010165007,2021S051310010165007,49.0,24.0,22.0,0.0071,0,10,7,1,...,165,7,10,519,519,1,170.02,4738,792,9910
498783,10010165006,2021S051310010165006,270.0,123.0,118.0,0.3063,0,10,6,1,...,165,7,10,519,519,1,170.02,4738,792,9910
498784,10010165002,2021S051310010165002,21.0,9.0,9.0,0.0084,0,10,2,1,...,165,7,10,519,519,1,170.02,4738,792,9910


In [21]:
print(MERGED_DF_CCB_PEI.crs)
MERGED_DF_CCB_PEI.head(3)

epsg:3979


C:\Users\daniel.kim\AppData\Local\Temp\2\ipykernel_9148\3032395524.py:1: FutureWarning: Accessing CRS of a GeoDataFrame without a geometry column is deprecated and will be removed in GeoPandas 0.12. Use GeoDataFrame.set_geometry to set the active geometry column.
  print(MERGED_DF_CCB_PEI.crs)


,id,DBUID,DGUID_x,DBRPLAMX_x,DBRPLAMY_x,LANDAREA_x,PRUID_x,geometry_x,OVERLAY_AREA,DGUID_y,DBRPLAMX_y,DBRPLAMY_y,LANDAREA_y,PRUID_y,geometry_y,PEI_TOTAL_AREA,NON_ECOPIA_AREA,PERCENTAGE_NOT_CONTAINING_ECOPIA,PERCENTAGE_CONTAINING_ECOPIA
0,1.0,11030266006,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335536.420 305604.721, 2335509.892 ...",643844.200397,2021S051311030266006,8.318885e+06,1.625373e+06,0.6305,11,"POLYGON ((2335549.465 305570.376, 2335536.420 ...",643844.200397,2.328306e-10,3.616258e-16,1.0
1,1.0,11030175002,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341238.622 306007.263, 2341189.666 ...",120420.969053,2021S051311030175002,8.324071e+06,1.625164e+06,0.1058,11,"POLYGON ((2341332.671 305999.874, 2341238.622 ...",120420.969053,5.820766e-11,4.833681e-16,1.0
2,1.0,11020453003,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346317.175 305128.782, 2346293.754 ...",931965.377813,2021S051311020453003,8.329620e+06,1.624529e+06,0.9123,11,"POLYGON ((2346319.460 305116.202, 2346317.175 ...",931965.377813,0.000000e+00,0.000000e+00,1.0


In [ ]:
# Calculating the population, total dwellings, and dwellings occupied by usual residents in both the Ecopia and non-Ecopia areas
# - multiply each of these values by the percentage of area overlap

# population

# total dwellings

# dwellings